# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Открытие и изучение файлов

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats as st
import re
import math
from IPython.display import display
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head(10))
display(df.info())
display(df.describe().T)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


## Разбейте данные на выборки

In [3]:
df_train, df_valid = train_test_split(df, test_size=0.30, random_state=12345) # разобьем данные на три воборки в соотношении 3:1:1
df_valid, df_test = train_test_split(df_valid,test_size = 0.5, random_state=12345)
print('Размер обучающей выборки:', df_train.shape[0],'объектов')
print('Размер валидационной выборки:', df_valid.shape[0],'объекта')
print('Размер тестовой выборки:',df_test.shape[0],'объекта')

Размер обучающей выборки: 2249 объектов
Размер валидационной выборки: 482 объекта
Размер тестовой выборки: 483 объекта


In [4]:

features = df.drop(['is_ultra'], axis=1) #Переменный признак
target = df['is_ultra'] #Целевой признак


features_valid = df_valid.drop(['is_ultra'], axis=1)#Валидационная выборка
target_valid = df_valid['is_ultra']


features_train = df_train.drop(['is_ultra'], axis=1)#Обучающая выборка
target_train = df_train['is_ultra']


features_test = df_test.drop(['is_ultra'], axis=1)#Тестовая выборка
target_test = df_test['is_ultra']

## Исследуйте модели

### Модель решающее дерево

In [5]:
for depth in range(1, 6): #Посмортим глубину дерева от 1 до 5
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)

    predictions_valid = model.predict(features_valid)

    print("Глубина дерева =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid)) 

Глубина дерева = 1 : 0.7510373443983402
Глубина дерева = 2 : 0.7800829875518672
Глубина дерева = 3 : 0.7863070539419087
Глубина дерева = 4 : 0.7883817427385892
Глубина дерева = 5 : 0.7614107883817427


Самая высокая  Accuracy 0.788 при глубине дерева 4

### Модель Случайный лес

In [6]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 5): #Зададим количество деревьев от 10 до 50 с шагом в 5, и глубину дерева с 1 до 10
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)

Accuracy наилучшей модели на валидационной выборке: 0.8132780082987552 Количество деревьев: 30 Максимальная глубина: 10


<div class="alert alert-success">
<b>Комментарий ревьюера</b>

Лес получше дерева :)
</div>

### Модель Логистическая регрессия

In [7]:
model = LogisticRegression(random_state=12345) 
model.fit(features_train, target_train) 
result = model.score(features_valid, target_valid) 
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.6950207468879668


## Проверьте модель на тестовой выборке

Т.к. самая высокая Accuracy у модели случайный лес с количеством деревьев 30 и глубиной 10, проверим ее на тестовой выборке

In [8]:
model = RandomForestClassifier(random_state=12345, n_estimators=30, max_depth=10)
model.fit(features_train, target_train) 
predictions_test = model.predict(features_test)
result_test = accuracy_score(target_test, predictions_test)
print("Accuracy модели случайный лес на тестовой выборке:", result_test)

Accuracy модели случайный лес на тестовой выборке: 0.8115942028985508


## (бонус) Проверьте модели на адекватность

In [9]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
dummy_clf.predict(features_train)
result_dummy = dummy_clf.score(features_train, target_train)
print("Accuracy модели фиктивный классификатор на тренировачной выборке выборке:", result_dummy)

Accuracy модели фиктивный классификатор на тренировачной выборке выборке: 0.6927523343708315


0,692 все что выше этого значения является адекватностью выбора модели, получается даже линейная регрессия, адекватна, если в условиях задачи была скорость обработки, линейная ригрессия имела место быть, но у нас нет такой задачи, значит все-таки случайный лес победил